In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
downloaded = drive.CreateFile({'id':'1X_W2WCt0sK4LV8b3SsNpRrechWUWMXub'}) # replace the id with id of file you want to access
downloaded.GetContentFile('orders.csv') 

NameError: ignored

In [3]:
downloaded = drive.CreateFile({'id':'1PLsiKpojrtg5cZBPAbOTOSy7xT7XSBRU'}) # replace the id with id of file you want to access
downloaded.GetContentFile('order_products__prior.csv') 

NameError: ignored

In [0]:
downloaded = drive.CreateFile({'id':'1sMdw_VtoRe2JkOp14EKH0UPuOgJE_ZKY'}) # replace the id with id of file you want to access
downloaded.GetContentFile('aisles.csv') 

In [0]:

downloaded = drive.CreateFile({'id':'1gYaZMuFCjkQ5WAuBJuddSiDXFFGkuzJh'}) # replace the id with id of file you want to access
downloaded.GetContentFile('order_products__train.csv') 

In [0]:
downloaded = drive.CreateFile({'id':'1BxZfBX41s1W7-BAZFHES0YeibmhCAKtZ'}) # replace the id with id of file you want to access
downloaded.GetContentFile('products.csv') 

In [0]:
downloaded = drive.CreateFile({'id':'1tSpbqCl_cuZwXLosSxhmJxUmJo58CyuL'}) # replace the id with id of file you want to access
downloaded.GetContentFile('sample_submission.csv') 

In [4]:
downloaded = drive.CreateFile({'id':'18-9dw53_Yfpfxi1rKZnLKO9kRIKOB25B'}) # replace the id with id of file you want to access
downloaded.GetContentFile('departments.csv') 

NameError: ignored

In [0]:
import numpy as np
import pandas as pd

In [0]:
orders= pd.read_csv("orders.csv", usecols=["order_id","user_id","order_number"])

Since the objective is to predict which previously purchased products will be in next order, let us first get the list of all products purchased by the customer.

In [0]:
# read the prior order file 
prior= pd.read_csv("order_products__prior.csv")
print(prior.shape)

(32434489, 4)


In [0]:
# merge with the orders file to get the user_id 
prior= pd.merge(prior, orders, how="inner", on="order_id")
print(prior.shape)

(32434489, 6)


In [0]:
# get the products and reorder status of the latest purchase of each user 
prior_grouped_df = prior.groupby("user_id")["order_number"].aggregate("max").reset_index()
print(prior_grouped_df.shape)

prior_df_latest = pd.merge(prior, prior_grouped_df, how="inner", on=["user_id", "order_number"])
print(prior_df_latest.shape)

prior_df_latest = prior_df_latest[["user_id", "product_id", "reordered"]]
print(prior_df_latest.shape)

prior_df_latest.columns = ["user_id", "product_id", "reordered_latest"]

(206209, 2)
(2139788, 6)
(2139788, 3)


In [0]:
# get the count of each product and number of reorders by the customer 
prior= prior.groupby(["user_id","product_id"])["reordered"].aggregate(["count", "sum"]).reset_index()
print(prior.shape)

prior.columns = ["user_id", "product_id", "reordered_count", "reordered_sum"]
print(prior.shape)


(13307953, 4)
(13307953, 4)


In [0]:
# merge the prior df with latest df 
prior = pd.merge(prior, prior_df_latest, how="left", on=["user_id","product_id"])
print(prior.shape)

(13307953, 5)


Read the train and test dataset and then merge with orders data to get the user_id for the corresponding order_id.

In [0]:
orders.drop(["order_number"],axis=1,inplace=True)
print(orders.shape)

(3421083, 2)


In [0]:
train_df = pd.read_csv("order_products__train.csv", usecols=["order_id"])
print(train_df.shape)

train_df = train_df.groupby("order_id").aggregate("count").reset_index()
print(train_df.shape)

test_df = pd.read_csv("sample_submission.csv", usecols=["order_id"])
print(test_df.shape)

train_df = pd.merge(train_df, orders, how="inner", on="order_id")
print(train_df.shape)

test_df = pd.merge(test_df, orders, how="inner", on="order_id")
print(test_df.shape)

print(train_df.shape, test_df.shape)

(1384617, 1)
(131209, 1)
(75000, 1)
(131209, 2)
(75000, 2)
(131209, 2) (75000, 2)


Merge the train and test data with prior to get the products purchased previously by the customer.

In [0]:
train_df = pd.merge(train_df, prior, how="inner", on="user_id")
print(train_df.shape)

test_df = pd.merge(test_df, prior, how="inner", on="user_id")
print(test_df.shape)

del prior, prior_grouped_df, prior_df_latest
print(train_df.shape, test_df.shape)

(8474661, 6)
(4833292, 6)
(8474661, 6) (4833292, 6)


products.csv file information about the products such as which department and aisle the given product belongs to. So merge the train and test data with product information.

In [0]:
products_df = pd.read_csv("products.csv", usecols=["product_id", "aisle_id", "department_id"])
print(products_df.shape)

train_df = pd.merge(train_df, products_df, how="inner", on="product_id")
print(train_df.shape)

test_df = pd.merge(test_df, products_df, how="inner", on="product_id")
print(test_df.shape)

del products_df
print(train_df.shape, test_df.shape)

(49688, 3)
(8474661, 8)
(4833292, 8)
(8474661, 8) (4833292, 8)


Now we have all the products that has been purchased previously by the customer along with some characteristics / featrures. So we can use the train dataset to populate the target variable i.e., whether the product has been reordered in the next order. 

In [0]:
train_y_df = pd.read_csv("order_products__train.csv", usecols=["order_id", "product_id", "reordered"])
print(train_y_df.shape)

train_y_df = pd.merge(train_y_df, orders, how="inner", on="order_id")
print(train_y_df.shape)

train_y_df = train_y_df[["user_id", "product_id", "reordered"]]


train_df = pd.merge(train_df, train_y_df, how="left", on=["user_id", "product_id"])
print(train_df.shape)

train_df["reordered_latest"].fillna(0, inplace=True)
print(train_df.shape)


del train_y_df

(1384617, 3)
(1384617, 4)
(8474661, 9)
(8474661, 9)


In [0]:
# target variable for train set 
train_y = train_df.reordered.values

# dataframe for test set predictions 
out_df = test_df[["order_id", "product_id"]]

# drop the unnecessary columns 
train_df = train_df.drop(["order_id", "user_id", "reordered"], axis=1)
test_df = test_df.drop(["order_id", "user_id"], axis=1)
print(train_df.shape, test_df.shape)

(8474661, 6) (4833292, 6)


In [0]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [0]:
xtrain,xtest, ytrain, ytest = train_test_split(train_df,train_y,test_size=0.20,random_state=42)

In [0]:
print(xtrain.shape)
print(xtest.shape)
print(ytrain.shape)
print(ytest.shape)


(6779728, 6)
(1694933, 6)
(6779728,)
(1694933,)


In [0]:
feature_columns = []
for feature_name in ["product_id","reordered_count","reordered_sum","reordered_latest","aisle_id","department_id"]:
    feature_columns.append(tf.feature_column.numeric_column(feature_name,
                                           dtype=tf.float32))

In [0]:
feature_columns

[NumericColumn(key='product_id', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='reordered_count', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='reordered_sum', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='reordered_latest', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='aisle_id', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='department_id', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [0]:
import numpy as np

In [1]:
NUM_EXAMPLES = len(train_df)

def make_input_fn(X, y, n_epochs=None, shuffle=True):
    def input_fn():
        dataset = tf.data.Dataset.from_tensor_slices((X.to_dict(orient='list'), y))
        if shuffle:
            dataset = dataset.shuffle(NUM_EXAMPLES)
    # For training, cycle thru dataset as many times as need (n_epochs=None).
        dataset = (dataset
          .repeat(n_epochs)
          .batch(NUM_EXAMPLES))
        return dataset
    return input_fn

# Training and evaluation input functions.
train_input_fn = make_input_fn(xtrain, ytrain)
eval_input_fn = make_input_fn(xtest, ytest, shuffle=False, n_epochs=1)
eval_input_fn2 = make_input_fn(test_df, np.zeros([1,4833292]).ravel(), shuffle=False, n_epochs=1)

NameError: ignored

In [0]:
params = {
  'n_trees': 50,
  'max_depth': 3,
  'n_batches_per_layer': 1,
  'center_bias': True
}

est = tf.estimator.BoostedTreesClassifier(feature_columns, **params)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpoxfr4n7_', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1e92a30d68>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
# Train model.
est.train(train_input_fn, max_steps=100)

# Evaluation.
results = est.evaluate(eval_input_fn)
clear_output()
pd.Series(results).to_frame()

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpoxfr4n7_/model.ckpt.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'


InvalidArgumentError: ignored

In [0]:
pred_dicts = list(est.predict(eval_input_fn2))
probs = pd.Series([pred['probabilities'][1] for pred in pred_dicts])


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp42ej02iy/model.ckpt-0
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [0]:
pred=probs.values

In [0]:
out_df

,order_id,product_id
0,17,1283
1,657743,1283
2,834223,1283
3,1446886,1283
4,1518981,1283
5,1559724,1283
6,1575667,1283
7,2290723,1283
8,2829393,1283
9,17,6291


In [0]:
cutoff = 0.2
pred[pred>=cutoff] = 1
pred[pred<cutoff] = 0
out_df["Pred"] = pred
out_df1 = out_df.ix[out_df["Pred"].astype('int')==1]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """


In [0]:
out_df1

,order_id,product_id,Pred
0,17,1283,1.0
1,657743,1283,1.0
2,834223,1283,1.0
3,1446886,1283,1.0
4,1518981,1283,1.0
5,1559724,1283,1.0
6,1575667,1283,1.0
7,2290723,1283,1.0
8,2829393,1283,1.0
9,17,6291,1.0


In [0]:
# when there are more than 1 product, merge them to a single string #
def merge_products(x):
    return " ".join(list(x.astype('str')))
out_df1 = out_df1.groupby("order_id")["product_id"].aggregate(merge_products).reset_index()
out_df1.columns = ["order_id", "products"]

In [0]:
out_df1

,order_id,products
0,17,1283 6291 7035 11494 13107 13535 15613 16965 1...
1,34,7035 21463 39275 47766 651 2361 2596 4031 5134...
2,137,21903 5134 311 1335 2078 2326 2661 3951 5025 5...
3,182,21903 39275 47766 9337 12456 13176 27156 37687...
4,257,47766 12456 13176 21137 39475 24852 27104 1442...
5,313,13535 21903 39275 12409 23537 16797 24852 2589...
6,353,21137 16797 33000 15541 17302 21573 22897 3556...
7,386,13535 21903 47766 21137 27156 39180 42265 2326...
8,414,21709 38777 44632 49191 33198 24838 25659 5322...
9,418,47766 44910 28934 12036 30391 47042 1025 4605 ...
